In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

try:
    WORKING_DIR
except NameError:
    WORKING_DIR = Path.cwd().parent.parent

%cd $WORKING_DIR
%pwd

In [ ]:
import numpy as np
from pymoo.indicators.hv import HV
import pfevaluator
import pandas as pd

from notebooks.utils import *

In [ ]:
volume_metrics = ["HV"]
hv_slide = 100
moo_metrics = list()
datasets = ["adult", "dutch", "credit", "compas"]
# datasets = ["adult"]

# The maximum single point
worse_recon_error_dict = dict(adult=1, dutch=1, credit=3, compas=1)
worse_prop_val_dict = dict(leakage=1, discrimination=1)
for dataset in datasets:
    path = f"results/alpha-beta-tradeoff/{dataset}/best.tinydb"
    print(path)
    ab_table = AlphaBetaTradeoffTable([path])

    cpf, cfb, ibsi, cpfsi = ab_table.get_tables(
        aggregate=False,
        latent_dim=32,
    )

    for property_ in ("discrimination", "leakage"):
        cpf, cfb, ibsi, cpfsi = remove_stratified_metrics(cpf, cfb, ibsi, cpfsi)
        results = [("CPF", cpf), ("CFB", cfb), ("IBSI", ibsi), ("CPFSI", cpfsi)]
        model_solutions = dict()
        for model, result in results:
            if not result.index.name == "estimator":
                result.set_index("estimator", inplace=True)

            solutions = dict()
            for estimator in ["lr", "rf"]:
                res = result.loc[estimator, :]
                table = pivot_table(res)
                solution_table = get_solutions(table, property_)
                solutions[estimator] = solution_table
            assert all(solutions["lr"].index == solutions["rf"].index)

            expansions = list()
            for idx in solutions["lr"].index:
                columns = solutions["lr"].columns
                index_names = solutions["lr"].index.names
                t1 = solutions["lr"].loc[idx].tolist()
                t2 = solutions["rf"].loc[idx].tolist()
                if property_ == "discrimination":
                    t1 = [t1[0], [t1[1], t1[2]]]
                    t2 = [t2[0], [t2[1], t2[2]]]
                expansion = expand_solutions(t1, t2)
                if property_ == "discrimination":
                    expansion = [flatten(t) for t in expansion]
                tmp = pd.DataFrame(expansion, columns=columns)
                for col, val in zip(index_names, idx):
                    tmp[col] = val
                tmp.set_index(index_names, inplace=True)
                expansions.append(tmp)
            expanded_solutions = pd.concat(expansions)
            model_solutions[model] = expanded_solutions

        all_solution_table = pd.concat(list(model_solutions.values()))
        display(all_solution_table.max())
        matrix_fitness = all_solution_table.to_numpy()
        reference_front = pfevaluator.find_reference_front(matrix_fitness)

        ref_point = np.array([worse_recon_error_dict[dataset], worse_prop_val_dict[property_], 1.0])
        for model, model_solution_table in model_solutions.items():
            for key, group in model_solution_table.groupby("seed"):
                data = group.to_numpy()

                with hidden_prints():
                    volume_metrics_values = pfevaluator.metric_volume(
                        hv_slide + data,
                        reference_front=reference_front,
                        metrics=volume_metrics,
                        hv_point=ref_point,
                    )
                
                hv = HV(ref_point=ref_point)

                nondom_data = pareto.eps_sort(data)
                c1r_value = c1r(nondom_data, reference_front)
                c2r_value = c2r(nondom_data, reference_front)
                hv_value = hv(np.array(nondom_data))

                entry = dict(
                    Dataset=dataset,
                    Property=property_,
                    Model=model,
                    seed=key,
                    c1r=c1r_value,
                    c2r=c2r_value,
                    hv=hv_value,
                    **volume_metrics_values,
                )

                moo_metrics.append(entry)

In [ ]:
to_drop = ["seed", "HV"]
# pd.set_eng_float_format(accuracy=2, use_eng_prefix=True)
pd.options.display.float_format = '{:,.2f}'.format
df = pd.DataFrame(moo_metrics).drop(to_drop, axis=1)
df = df.groupby(["Dataset", "Property", "Model"], sort=False).agg([np.mean, stderr])
(df * 100).round(2)